# Laboratorio 1 - Detección de Pishing

Hugo Román

Roberto Castillo

In [1]:
#Importar las librerias

import pandas as pd
import numpy as np
import string
import pandas_profiling as profiling
from pandas_profiling import ProfileReport

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_114/1364463717.py:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as profiling


### Parte 1 - Ingeniería de características

In [2]:
#Cargando dataset
df = pd.read_csv('dataset_pishing.csv')
#Mostrando 5 observaciones
df.sample(n=5).head() # print a random sample of the DataFrame


,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
6971,https://www.bodemerauto.com/,0,1,0,0,0,0,0,0,0,...,0,1,0,136,2786,702387,0,0,3,legitimate
1933,https://www.duvolab.com/,0,1,0,0,0,0,0,0,0,...,1,0,0,351,1109,5582632,0,0,2,legitimate
327,http://uniteddental.ca/new/trust/db/,0,0,0,0,0,0,0,0,0,...,1,1,0,1191,6478,0,0,1,1,phishing
2292,https://nsp.biz.ua/registraciya-v-nsp,0,0,0,0,0,0,0,0,0,...,1,0,0,3176,3911,7910131,0,0,2,phishing
2738,https://www.zabor-vn.com/system/csvprice_pro/s...,0,1,0,0,0,0,0,0,0,...,1,1,0,120,1706,4830221,0,1,2,phishing


In [3]:
#Cantidad de etiquetas de la columna status
df['status'].value_counts()


legitimate    5715
phishing      5715
Name: status, dtype: int64

Como se puede observar en los valores de arriba, la columna status presenta valores balanceados

In [4]:
#Separando dominio
df['domainS']= df["url"].str.split("/",expand = False)
df['Domain'] = df['domainS'].apply(lambda x: x[2])
df[['url', 'Domain']].head(5)


,url,Domain
0,http://www.crestonwood.com/router.php,www.crestonwood.com
1,http://shadetreetechnology.com/V4/validation/a...,shadetreetechnology.com
2,https://support-appleld.com.secureupdate.duila...,support-appleld.com.secureupdate.duilawyeryork...
3,http://rgipt.ac.in,rgipt.ac.in
4,http://www.iracing.com/tracks/gateway-motorspo...,www.iracing.com


In [5]:
#En base al artículo “Towards Benchmark Datasets for ML Based Wensite Phishing Detection: An
#Experimental Study”, derivar las características basadas en el dominio:

#F1-F2
df['URL_length'] = df['url'].apply(len)
df['Domain_Length'] = df['Domain'].apply(len)
df[['url', 'URL_length','Domain','Domain_Length']].head(5)

,url,URL_length,Domain,Domain_Length
0,http://www.crestonwood.com/router.php,37,www.crestonwood.com,19
1,http://shadetreetechnology.com/V4/validation/a...,77,shadetreetechnology.com,23
2,https://support-appleld.com.secureupdate.duila...,126,support-appleld.com.secureupdate.duilawyeryork...,50
3,http://rgipt.ac.in,18,rgipt.ac.in,11
4,http://www.iracing.com/tracks/gateway-motorspo...,55,www.iracing.com,15


In [6]:
#F4-F20
df['SpecialCharacters']= df.apply(lambda p: sum( not q.isalpha() for q in p["url"] ), axis=1)
df[['url', 'SpecialCharacters']].head(5)

,url,SpecialCharacters
0,http://www.crestonwood.com/router.php,7
1,http://shadetreetechnology.com/V4/validation/a...,24
2,https://support-appleld.com.secureupdate.duila...,38
3,http://rgipt.ac.in,5
4,http://www.iracing.com/tracks/gateway-motorspo...,10


In [7]:
#F25
df['Tokens'] = (df['url'].str.contains('https')).any()
string = "https"
def domain(url):
    if url[:5] == string:
        return "isHttps"
    return False
df['Tokens'] = df['url'].apply(domain)
df[['url', 'Tokens']].head(5)


,url,Tokens
0,http://www.crestonwood.com/router.php,False
1,http://shadetreetechnology.com/V4/validation/a...,False
2,https://support-appleld.com.secureupdate.duila...,isHttps
3,http://rgipt.ac.in,False
4,http://www.iracing.com/tracks/gateway-motorspo...,False


In [8]:
#F26-27
df['Digits_URL'] = df['url'].str.count('\d')
df['Digits_Domain'] = df['Domain'].str.count('\d')
df[['url', 'Digits_URL','Domain','Digits_Domain']].head(5)

,url,Digits_URL,Domain,Digits_Domain
0,http://www.crestonwood.com/router.php,0,www.crestonwood.com,0
1,http://shadetreetechnology.com/V4/validation/a...,17,shadetreetechnology.com,0
2,https://support-appleld.com.secureupdate.duila...,19,support-appleld.com.secureupdate.duilawyeryork...,0
3,http://rgipt.ac.in,0,rgipt.ac.in,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,www.iracing.com,0


In [9]:
#Mostrando columnas nuevas
df[['url','Domain', 'URL_length','Domain_Length','SpecialCharacters','Tokens','Digits_URL','Digits_Domain']].head(5)

,url,Domain,URL_length,Domain_Length,SpecialCharacters,Tokens,Digits_URL,Digits_Domain
0,http://www.crestonwood.com/router.php,www.crestonwood.com,37,19,7,False,0,0
1,http://shadetreetechnology.com/V4/validation/a...,shadetreetechnology.com,77,23,24,False,17,0
2,https://support-appleld.com.secureupdate.duila...,support-appleld.com.secureupdate.duilawyeryork...,126,50,38,isHttps,19,0
3,http://rgipt.ac.in,rgipt.ac.in,18,11,5,False,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,www.iracing.com,55,15,10,False,0,0


In [10]:
#Eliminando dominio 
df.drop('domainS', inplace=True, axis=1)
df.drop('Domain', inplace=True, axis=1)

#Variable status a binaria
#Legitimo = 0
#Phising = 1
df['status'].replace(['legitimate', 'phishing'],[0, 1], inplace=True)

In [11]:
#Mostrando dataset final
df.sample(n=5).head() # print a random sample of the DataFrame

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,dns_record,google_index,page_rank,status,URL_length,Domain_Length,SpecialCharacters,Tokens,Digits_URL,Digits_Domain
927,http://www.madehow.com/Volume-5/Computer-Mouse...,0,1,1,0,0,0,0,1,0,...,0,0,5,0,51,15,11,False,1,0
9636,https://www.seatguru.com/airlines/LATAM_Chile/...,0,1,0,0,0,0,0,0,0,...,0,0,5,0,61,16,10,isHttps,0,0
960,https://www.xhkzv.com/,0,1,0,0,0,0,0,0,0,...,0,0,1,0,22,13,6,isHttps,0,0
10916,http://perron.team/1ndex.php,0,0,0,0,0,0,0,0,0,...,0,1,0,1,28,11,7,False,1,0
4731,https://www.likealocalguide.com/freiburg/compa...,0,1,1,0,0,0,0,1,0,...,0,0,5,0,69,23,10,isHttps,0,0


In [12]:
#Reporte
prof = ProfileReport(df, minimal = True)
prof.to_file(output_file='output.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 91.76it/s]


In [13]:
#Clean Data
df.drop('sfh', axis=1)
df.drop('ratio_nullHyperlinks', inplace=True, axis=1)
df.drop('ratio_intRedirection', inplace=True, axis=1)
df.drop('ratio_intErrors', inplace=True, axis=1)
df.drop('submit_email', inplace=True, axis=1)


## Parte 2 – Implementación 

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   ip                          11430 non-null  int64  
 2   nb_www                      11430 non-null  int64  
 3   nb_com                      11430 non-null  int64  
 4   nb_dslash                   11430 non-null  int64  
 5   http_in_path                11430 non-null  int64  
 6   punycode                    11430 non-null  int64  
 7   port                        11430 non-null  int64  
 8   tld_in_path                 11430 non-null  int64  
 9   tld_in_subdomain            11430 non-null  int64  
 10  abnormal_subdomain          11430 non-null  int64  
 11  nb_subdomains               11430 non-null  int64  
 12  prefix_suffix               11430 non-null  int64  
 13  random_domain               114

In [15]:
# Obtenemos las columnas objeto
object_cols = [col for col in df.columns if df[col].dtype == "object"]
object_cols

['url', 'Tokens']

Notese que tenemos dos columnas obj (url y Tokens), esta nos dificultará el desempeño de ciertos modelos, por lo que la eliminaremos.

In [16]:
df = df.drop(['url', 'Tokens'], axis = 1).copy()

In [17]:
# Obtenemos X y Y
y = df['status']
X = df.drop('status',axis=1)
X.shape, y.shape

((11430, 66), (11430,))

Se hace la separación de train, test y validations

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.22, random_state=1) # 0.22 x 0.7 = 0.15
                                    
X_train.shape, X_test.shape, X_val.shape

((6240, 66), (3429, 66), (1761, 66))

## Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(max_depth=5)

forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=5)

In [20]:
# Se hacen las predicciones correspondientes
y_pred = forest.predict(X_test)
y_pred_val = forest.predict(X_val)

#Se obtiene el accuracy score
from sklearn.metrics import accuracy_score

print("Accuracy score of the model on test set: ", accuracy_score(y_test, y_pred))
print("Accuracy score of the model on validation set: ", accuracy_score(y_val, y_pred_val))

Accuracy score of the model on test set:  0.937007874015748
Accuracy score of the model on validation set:  0.9534355479840999


In [21]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix of the model on test set: \n", confusion_matrix(y_test, y_pred))
print("Confusion matrix of the model on validation set: \n", confusion_matrix(y_val, y_pred_val))


Confusion matrix of the model on test set: 
 [[1612   99]
 [ 117 1601]]
Confusion matrix of the model on validation set: 
 [[840  35]
 [ 47 839]]


In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix of the model on test set')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [23]:
cm = confusion_matrix(y_val, y_pred_val)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix of the model on validation set')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [33]:
from sklearn.metrics import recall_score, f1_score

print("Recall score of the model on test set: ", recall_score(y_test, y_pred))
print("Recall score of the model on validation set: ", recall_score(y_val, y_pred_val))

print("F1 score of the model on test set: ", f1_score(y_test, y_pred))
print("F1 score of the model on validation set: ", f1_score(y_val, y_pred_val))

Recall score of the model on test set:  0.9318975552968568
Recall score of the model on validation set:  0.9469525959367946
F1 score of the model on test set:  0.9368051492100644
F1 score of the model on validation set:  0.953409090909091


## Multilayer perceptron

In [25]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=0.001)

# fit the model 
mlp.fit(X_train, y_train)

MLPClassifier(alpha=0.001)

In [26]:
# Se hacen las predicciones y se obtienen los accuracy score
y_test_mlp = mlp.predict(X_test)
y_val_mlp = mlp.predict(X_val)

print("Accuracy score of the model on test set: ", accuracy_score(y_test, y_test_mlp))
print("Accuracy score of the model on validation set: ", accuracy_score(y_val, y_val_mlp))

Accuracy score of the model on test set:  0.8404782735491397
Accuracy score of the model on validation set:  0.8358886996024986


In [27]:
print("Confusion matrix of the model on test set: \n", confusion_matrix(y_test, y_test_mlp))
print("Confusion matrix of the model on validation set: \n", confusion_matrix(y_val, y_val_mlp))

Confusion matrix of the model on test set: 
 [[1337  374]
 [ 173 1545]]
Confusion matrix of the model on validation set: 
 [[682 193]
 [ 96 790]]


In [28]:
cm = confusion_matrix(y_test, y_test_mlp)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix of the model on test set')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [29]:
cm = confusion_matrix(y_val, y_val_mlp)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix of the model on validation set')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [32]:
print("Recall score of the model on test set: ", recall_score(y_test, y_test_mlp))
print("Recall score of the model on validation set: ", recall_score(y_val, y_val_mlp))

print("F1 score of the model on test set: ", f1_score(y_test, y_test_mlp))
print("F1 score of the model on validation set: ", f1_score(y_val, y_val_mlp))

Recall score of the model on test set:  0.8993015133876601
Recall score of the model on validation set:  0.891647855530474
F1 score of the model on test set:  0.8496013197690404
F1 score of the model on validation set:  0.8453718566078117


In [34]:

import pandas as pd

data = {'Accuracy score on test set': [accuracy_score(y_test, y_pred), accuracy_score(y_test, y_test_mlp)],
        'Accuracy score on validation set': [accuracy_score(y_val, y_pred_val), accuracy_score(y_val, y_val_mlp)],
        'Recall score on test set': [recall_score(y_test, y_pred), recall_score(y_test, y_test_mlp)],
        'Recall score on validation set': [recall_score(y_val, y_pred_val), recall_score(y_val, y_val_mlp)],
        'F1 score on test set': [f1_score(y_test, y_pred), f1_score(y_test, y_test_mlp)],
        'F1 score on validation set': [f1_score(y_val, y_pred_val), f1_score(y_val, y_val_mlp)]}

dfResults = pd.DataFrame(data, index=['Random Forest', 'Multilayer Perceptrons'])

dfResults

,Accuracy score on test set,Accuracy score on validation set,Recall score on test set,Recall score on validation set,F1 score on test set,F1 score on validation set
Random Forest,0.937008,0.953436,0.931898,0.946953,0.936805,0.953409
Multilayer Perceptrons,0.840478,0.835889,0.899302,0.891648,0.849601,0.845372


### Discusión

¿Cuál es el impacto de clasificar un sitio legítimo como Pishing?  

La clasificación de un sitio web auténtico como phishing  puede tener graves consecuencias, como el daño a la reputación del propietario del sitio web, la pérdida de ingresos, la confusión y molestia de los usuarios y el desperdicio de valiosos recursos. Un falso positivo puede hacer que el sitio web se bloquee o se marque como poco fiable, lo que puede reducir el tráfico y la confianza. Como resultado, los usuarios pueden llegar a confiar menos en el sistema de detección de phishing, así como quien lo haya desarrollado.

Cuál es el impacto de clasificar un sitio de Pishing como legítimo? 

Un falso negativo, a veces denominado clasificación de un sitio web de phishing como auténtico, puede tener efectos perjudiciales tanto para el propietario del sitio web como para los usuarios que puedan verse afectados por la falsa alarma. Los falsos negativos pueden permitir que los esfuerzos de phishing pasen desapercibidos y conduzcan al robo de datos críticos, incluidas contraseñas e información financiera. De ello pueden derivarse robos de identidad, pérdidas de dinero y otros perjuicios para las personas afectadas. Los falsos negativos también pueden dañar la fe de los consumidores en la herramienta de detección de phishing y en la empresa que la ha desarrollado, haciéndoles más propensos a desatender o ignorar las advertencias en el futuro. Los falsos negativos también pueden provocar una pérdida de recursos y productividad, ya que la organización puede tener que dedicar tiempo y dinero a hacer frente a las consecuencias de un ataque de phishing con éxito.

En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares 
de clasificación de pishing? 

Para equilibrar la compensación entre falsos positivos y falsos negativos, eligiría la puntuación F1, que es la media armónica de la precisión y la recuperación. Una puntuación F1 alta significa que el modelo puede detectar de forma fiable sitios de phishing al tiempo que reduce el número de falsos positivos y falsos negativos. También significa que el modelo tiene una buena precisión y recuperación.


¿Qué modelo es mejor para la clasificación de Pishing? Justifique

La precisión del modelo Random Forest es de 0,95, lo que significa que puede clasificar con precisión el 95% de los casos del conjunto de pruebas. Con una puntuación Recall de 0,94, el modelo identificó con éxito el 94% de los casos positivos del conjunto de prueba (es decir, sitios de phishing). Dado que la puntuación F1 es una media ponderada de las puntuaciones de precisión y recuperación, la puntuación F1 del modelo de 0,95 muestra que existe un equilibrio razonable entre ambas.

Por otro lado, el modelo Multilayer Perceptrón tiene una precisión inferior, de 0,84, y sólo puede clasificar con precisión el 84% de los casos del conjunto de prueba. En comparación con el modelo Random Forest, la puntuación de recuperación del modelo de 0,89 muestra que identificó correctamente el 89% de los casos positivos. El modelo presenta un peor equilibrio entre precisión y recuperación que el modelo Random Forest, según la puntuación F1 de 0,84.

Por lo tanto, parece que el modelo Random Forest es más adecuado para clasificar el phishing basándose en los datos obtenidos. El tamaño y la calidad de los datos de entrenamiento, las características empleadas y los hiperparámetros del modelo son algunos ejemplos de otras variables que pueden influir en el rendimiento de los modelos. Para llegar a una conclusión firme, sería necesario un análisis y una comparación más completos de los modelos.


En base a las métricas obtenidas ¿es necesaria la intervención de una persona humana para 
tomar la decisión final? 

Incluso con el uso de algoritmos de aprendizaje automático, es factible, basándose en las métricas obtenidas, que siga siendo necesaria la intervención humana para tomar la decisión final. Aunque son deseables y sugieren un rendimiento superior, las puntuaciones altas de precisión, recuperación y F1 no garantizan una ejecución impecable. La elección final puede seguir requiriendo la habilidad y el juicio humanos en los casos en que las predicciones del modelo sigan siendo ambiguas o imprecisas. Se necesita un experto humano para detectar y evaluar adecuadamente algunos ataques de phishing, ya que pueden ser complejos y evasivos. Para obtener los mejores resultados, es crucial contar con un enfoque exhaustivo que incorpore también la interacción y la experiencia humanas, aunque los algoritmos de aprendizaje automático puedan ser una herramienta beneficiosa para la detección del phishing.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a5a7a80e-262f-4d26-99b0-d8ffdbda695a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>